Let's load up some Juniper config and parse it into a YANG data model.  First, we'll read the file.

In [1]:
from ntc_rosetta import get_driver
import json

junos = get_driver("junos", "openconfig")
junos_driver = junos()

# Strip any rpc tags before and after `<configuration>...</configuration>`
with open("../../tests/models/openconfig/data/openconfig-vlan/parse/junos/config/dev_conf", "r") as fp:
    config = fp.read()
print(config)

<configuration>
        <interfaces>
            <interface>
                <name>xe-0/0/0</name>
                <unit>
                    <name>0</name>
                    <description>this is interface xe-0/0/0</description>
                </unit>
            </interface>
            <interface>
                <name>xe-0/0/1</name>
                <unit>
                    <name>0</name>
                    <family>
                        <ethernet-switching>
                            <interface-mode>access</interface-mode>
                            <vlan>
                                <members>10</members>
                            </vlan>
                        </ethernet-switching>
                    </family>
                </unit>
            </interface>
            <interface>
                <name>xe-0/0/3</name>
                <unit>
                    <name>0</name>
                    <family>
                        <ethernet-switching>
              

Now, we parse the data and take a look at the data model.

In [2]:
from sys import exc_info
from yangson.exceptions import SemanticError

try:
    parsed = junos_driver.parse(
        native={"dev_conf": config},
        validate=False,
        include=[
            "/openconfig-interfaces:interfaces",
            "/openconfig-network-instance:network-instances/network-instance/name",
            "/openconfig-network-instance:network-instances/network-instance/config",
            "/openconfig-network-instance:network-instances/network-instance/vlans",        
        ]
    )
except SemanticError as e:
    print(f"error: {e}")

print(json.dumps(parsed.raw_value(), indent=2))

{
  "openconfig-interfaces:interfaces": {
    "interface": [
      {
        "name": "xe-0/0/0",
        "config": {
          "name": "xe-0/0/0",
          "type": "iana-if-type:ethernetCsmacd",
          "enabled": true
        },
        "subinterfaces": {
          "subinterface": [
            {
              "index": 0,
              "config": {
                "index": 0,
                "description": "this is interface xe-0/0/0"
              }
            }
          ]
        }
      },
      {
        "name": "xe-0/0/1",
        "config": {
          "name": "xe-0/0/1",
          "type": "iana-if-type:ethernetCsmacd",
          "enabled": true
        },
        "subinterfaces": {
          "subinterface": [
            {
              "index": 0,
              "config": {
                "index": 0
              }
            }
          ]
        },
        "openconfig-if-ethernet:ethernet": {
          "openconfig-vlan:switched-vlan": {
            "config": {
          

I bet you're thinking it would be cool if we could dump that YANG stuff into native configuration for, gasp, a different vendor!  Sure, we simply use the `translate` method with the appropriate driver.

In [3]:
ios = get_driver("ios", "openconfig")
ios_driver = ios()
native = ios_driver.translate(candidate=parsed.raw_value())

print(native)

interface xe-0/0/0
   no shutdown
   exit
!
interface xe-0/0/0.0
   description this is interface xe-0/0/0
   exit
!
interface xe-0/0/1
   no shutdown
   switchport mode access
   switchport access vlan 10
   exit
!
interface xe-0/0/3
   no shutdown
   switchport mode trunk
   switchport trunk allowed vlan 10,20
   exit
!
interface xe-0/0/4
   no shutdown
   switchport mode trunk
   switchport trunk allowed vlan 100,200
   exit
!
interface xe-0/0/5
   no shutdown
   switchport mode access
   switchport access vlan 100
   exit
!
vlan 1
   name default
   no shutdown
   exit
!
vlan 20
   name prod
   no shutdown
   exit
!
vlan 10
   shutdown
   exit
!
vlan 100
   name VLAN-100
   no shutdown
   exit
!
vlan 200
   name VLAN-200
   no shutdown
   exit
!



Pretty cool, right?!  Well, except those pesky interface names don't look quite right.  Let's take a look at fixing those up and how we can manipulate data model attributes.  To do that, we need to locate the attribute in the tree using the `parse_resource_id` method.

In [4]:
# Locate the data in the tree, vlan example
parsed.datamodel.parse_resource_id("openconfig-network-instance:network-instances/network-instance=0/vlans/vlan=VLAN-200")

In [11]:
# Locate the data in the tree, interface example. We need to modify this one
# Note that we have to URL-escape the forward slashes per https://tools.ietf.org/html/rfc8040#section-3.5.3
# This returns an "instance route" https://yangson.labs.nic.cz/instance.html?highlight=arrayentry#yangson.instance.InstanceRoute
irt = parsed.datamodel.parse_resource_id("openconfig-interfaces:interfaces/interface=xe-0%2F0%2F0")
current_data = parsed.root.goto(irt)
print("Current node configuration: ", json.dumps(current_data.raw_value(), indent=2))
modify_data = current_data.raw_value()
ifname = 'Ethernet0/0/1'
modify_data['name'] = ifname
modify_data['config']['name'] = ifname
modify_data['subinterfaces']['subinterface'][0]['config']['description'] = 'this is Ethernet0/0/1'
stub = current_data.update(modify_data, raw=True)
print("Candidate node configuration: ", json.dumps(stub.raw_value(), indent=2))

Current configuration:  {
  "name": "xe-0/0/0",
  "config": {
    "name": "xe-0/0/0",
    "type": "iana-if-type:ethernetCsmacd",
    "enabled": true
  },
  "subinterfaces": {
    "subinterface": [
      {
        "index": 0,
        "config": {
          "index": 0,
          "description": "this is interface xe-0/0/0"
        }
      }
    ]
  }
}
Candidate configuration:  {
  "name": "Ethernet0/0/1",
  "config": {
    "name": "Ethernet0/0/1",
    "type": "iana-if-type:ethernetCsmacd",
    "enabled": true
  },
  "subinterfaces": {
    "subinterface": [
      {
        "index": 0,
        "config": {
          "index": 0,
          "description": "this is Ethernet0/0/1"
        }
      }
    ]
  }
}


Cool, but what about the rest of the interfaces in the list?  Yangson provides an iterator for array nodes.

In [57]:
import re

irt = parsed.datamodel.parse_resource_id("openconfig-interfaces:interfaces/interface")
iface_objs = parsed.root.goto(irt)
p, sub = re.compile(r'xe-'), 'Ethernet'

for iface in iface_objs:
    data = iface.raw_value()
    data['name'] = p.sub(sub, data['name'])
    data['config']['name'] = p.sub(sub, data['config']['name'])
    iface = iface.update(data, raw=True)
    print(iface.value)

{'name': 'Ethernet0/0/0', 'config': {'name': 'Ethernet0/0/0', 'type': ('ethernetCsmacd', 'iana-if-type'), 'enabled': True}, 'subinterfaces': {'subinterface': [{'index': 0, 'config': {'index': 0, 'description': 'this is interface xe-0/0/0'}}]}}
{'name': 'Ethernet0/0/1', 'config': {'name': 'Ethernet0/0/1', 'type': ('ethernetCsmacd', 'iana-if-type'), 'enabled': True}, 'subinterfaces': {'subinterface': [{'index': 0, 'config': {'index': 0}}]}, 'openconfig-if-ethernet:ethernet': {'openconfig-vlan:switched-vlan': {'config': {'interface-mode': 'ACCESS', 'access-vlan': 10}}}}
{'name': 'Ethernet0/0/3', 'config': {'name': 'Ethernet0/0/3', 'type': ('ethernetCsmacd', 'iana-if-type'), 'enabled': True}, 'subinterfaces': {'subinterface': [{'index': 0, 'config': {'index': 0}}]}, 'openconfig-if-ethernet:ethernet': {'openconfig-vlan:switched-vlan': {'config': {'interface-mode': 'TRUNK', 'trunk-vlans': [10, 20]}}}}
{'name': 'Ethernet0/0/4', 'config': {'name': 'Ethernet0/0/4', 'type': ('ethernetCsmacd', 'i

In [59]:
native = ios_driver.translate(candidate=iface_objs.top().raw_value())

print(native)

interface xe-0/0/0
   no shutdown
   exit
!
interface xe-0/0/0.0
   description this is interface xe-0/0/0
   exit
!
interface xe-0/0/1
   no shutdown
   switchport mode access
   switchport access vlan 10
   exit
!
interface xe-0/0/3
   no shutdown
   switchport mode trunk
   switchport trunk allowed vlan 10,20
   exit
!
interface xe-0/0/4
   no shutdown
   switchport mode trunk
   switchport trunk allowed vlan 100,200
   exit
!
interface xe-0/0/5
   no shutdown
   switchport mode access
   switchport access vlan 100
   exit
!
vlan 1
   name default
   no shutdown
   exit
!
vlan 20
   name prod
   no shutdown
   exit
!
vlan 10
   shutdown
   exit
!
vlan 100
   name VLAN-100
   no shutdown
   exit
!
vlan 200
   name VLAN-200
   no shutdown
   exit
!

